# Asthma Dismissal Records

In [1]:
import os
import ast
import urllib2
import urllib
import json
import numpy as np
import shapely
import matplotlib.pyplot as pl
import geopandas as gpd
import pandas as pd

from cStringIO import StringIO
from geopandas import GeoDataFrame
from scipy.spatial.distance import pdist, squareform

%matplotlib inline

## Download using SQL

In [2]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql'

In [3]:
def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    """ Queries a url with an sql compatible search
    Code taken from Fbb55
    Arguments:
    query : the sql query string
    format : the expected format
    source " the url of the API
    """
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

In [4]:
facilityQuery = '''SELECT count(cartodb_id), zipcodes 
                  FROM NYCAsthmasDismissals 
                  GROUP BY zipcodes
                  '''

facilityStream = queryCartoDB(facilityQuery, format='JSON')

facilityData   = json.loads(facilityStream.read())
facilities     = pd.DataFrame.from_dict(facilityData['rows'])
facilities.columns = ['Counts','ZIPCODE']
print facilities.shape
facilities.head()

(29, 2)


,Counts,ZIPCODE
0,1699,10461
1,405,11373
2,1698,10451
3,220,11691
4,246,11375


## Shapefile for NYC Zipcode 

In [5]:
zips_shape = gpd.read_file(os.getenv("PUIDATA") + "/" + "ZIP_CODE_040114.shp")

In [6]:
zips_shape.columns

Index([      u'AREA',    u'BLDGZIP',     u'COUNTY',   u'CTY_FIPS',
       u'POPULATION',    u'PO_NAME', u'SHAPE_AREA',  u'SHAPE_LEN',
            u'STATE',    u'ST_FIPS',        u'URL',    u'ZIPCODE',
         u'geometry'],
      dtype='object')

In [7]:
zips_shape.shape

(263, 13)

In [8]:
zips_shape.to_crs(epsg=2263, inplace=True)

In [9]:
zips_shape.loc[:,'center'] = zips_shape.loc[:,'geometry'].centroid

In [10]:
zips_shape.drop([u'AREA',    u'BLDGZIP',     u'COUNTY',   u'CTY_FIPS',
           u'POPULATION',    u'PO_NAME', u'SHAPE_AREA',  u'SHAPE_LEN',
           u'STATE',    u'ST_FIPS',        u'URL', u'geometry'], 
          axis = 1, inplace = True)

In [12]:
zips_shape.ZIPCODE = zips_shape.ZIPCODE.astype('int64')
zips_shape.head()

,ZIPCODE,center
0,11436,POINT (1040696.262135346 185580.2219261284)
1,11213,POINT (1001912.871996465 183803.1131541364)
2,11212,POINT (1008365.375095716 180768.9448689773)
3,11225,POINT (997023.1926918954 180870.3533321856)
4,11218,POINT (990785.2911208547 173656.0483912209)
